In [102]:
import torch
from torch import nn

import numpy as np
import pandas as pd
from scipy.stats import norm
from urllib.request import urlopen

from torch.distributions.constraints import positive

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam, SGD
from pyro.contrib.oed.eig import marginal_eig
from pyro.nn.module import to_pyro_module_
import pyro.poutine as poutine
from pyro.contrib.util import lexpand
from pyro.nn import PyroModule, PyroSample, PyroParam

import os
import sys
parent_dir_abs = os.path.abspath(os.pardir)
sys.path.insert(0, parent_dir_abs) 
from pyro_oed_src import posterior_eig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import math

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f'Using device: {device}.')

Using device: cuda.


In [21]:
# thresholds for measuring data
designs = torch.tensor([0,2,4,6,8,10], device=device).reshape((6,1))

In [74]:
designs[0]

tensor(0, device='cuda:0')

In [59]:
lexpand(designs,1000,100).shape[:-1]


torch.Size([1000, 100])

In [75]:
lam_0 = torch.tensor(0.0, device=device)
lam_1 = torch.tensor(10.0, device=device)
n = torch.tensor(1000, device=device)

def model(d):
    with pyro.plate_stack("plate", d.shape[:-1]): 
        theta = pyro.sample("theta", dist.Uniform(lam_0,lam_1))
        return pyro.sample("m", dist.Poisson(theta))

In [109]:
class posterior_guide_nn(PyroModule):
    def __init__(self, in_features, lay1_out, lay2_out):
        super().__init__()
        self.a1 = nn.Linear(in_features, lay1_out).to(device)
        #self.a2 = nn.Linear(lay1_out, lay2_out).to(device)
        self.a3 = nn.Linear(lay2_out,1).to(device)

        self.b1 = nn.Linear(in_features, lay1_out).to(device)
        #self.b2 = nn.Linear(lay1_out, lay2_out).to(device)
        self.b3 = nn.Linear(lay2_out,1).to(device)
        
    def nn_a(self, input):
        z = nn.functional.relu(self.a1(input))
        #z = nn.functional.relu(self.a2(z))
        return self.a3(z)
    
    def nn_b(self, input):
        z = nn.functional.relu(self.b1(input))
        #z = nn.functional.relu(self.b2(z))
        return self.b3(z)
    
    def forward(self, y_dict, design, observation_labels, target_labels):

        m = y_dict["m"]
        input = torch.sum(m>= design).float()
        #print(input.unsqueeze(0).shape)
        
        alpha = self.nn_a(input.unsqueeze(0)).squeeze().to(device)
        beta = self.nn_b(input.unsqueeze(0)).squeeze().to(device)

        print("a", alpha, "b", beta)

        pyro.sample("theta", dist.AffineBeta(alpha.exp(), beta.exp(), 5, 5)) # scaled beta distribution inputs:(alpha, beta, loc, scale) f(X) = loc + scale*X

In [110]:
eigs = {}
full_history = []

pyro.set_rng_seed(6)
num_steps = 100

nn_guide = posterior_guide_nn(1, 8, 8)
for d in designs:
    pyro.clear_param_store()
    eig, history = posterior_eig(model,
                    d,                          # tensor of possible designs
                    "m",                            # observation labels
                    "theta",                 # target (parameter) labels
                    num_samples=400,                # number of samples to draw per step in the expectation
                    num_steps=num_steps,            # number of gradient steps
                    guide=nn_guide,                 # guide q(y)
                    optim=SGD({'lr': 5e-6}),      # optimizer 
                    eig=True,                       # compute the average posterior entropy (APE) instead of the EIG (if false)
                    return_history=True             # returns a tensor giving the loss function at each step of optimization
                    ) 
    full_history.append(history)
    eigs[times] = eig


full_history = torch.stack(full_history)

a tensor(1.6938, device='cuda:0', grad_fn=<SqueezeBackward0>) b tensor(74.6351, device='cuda:0', grad_fn=<SqueezeBackward0>)
a tensor(390.1102, device='cuda:0', grad_fn=<SqueezeBackward0>) b tensor(nan, device='cuda:0', grad_fn=<SqueezeBackward0>)


ValueError: Expected parameter concentration (Tensor of shape (2,)) of distribution Dirichlet(concentration: torch.Size([2])) to satisfy the constraint IndependentConstraint(GreaterThan(lower_bound=0.0), 1), but found invalid values:
tensor([inf, nan], device='cuda:0', grad_fn=<StackBackward0>)
Trace Shapes:    
 Param Sites:    
  a1$$$weight 8 1
    a1$$$bias   8
  a3$$$weight 1 8
    a3$$$bias   1
  b1$$$weight 8 1
    b1$$$bias   8
  b3$$$weight 1 8
    b3$$$bias   1
Sample Sites:    